In [31]:
# 输入数据的参数
_TRAINSET1='{"name":"trainset1","type":0,"uri":"tmp_f3e3796135a8457e90f87536d759881b"}'
_TRAINSET2='{"name":"trainset2","type":0,"uri":"tmp_a90dbcba9ae14afaa171ed071cd126ea"}'
_TESTSETA='{"name":"testsetA","type":0,"uri":"tmp_c4e991086e684bddafeaac9de2666e07"}'
_TESTSETB='{"name":"testsetB","type":0,"uri":"tmp_03d402d824cf42e582b6646f7db46700"}'

# 输出数据的参数
_OUTPUT='[{"name":"output1","type":0,"uri":"tmp_79a44162f38f41738d6e4f577ef9c56f"},{"name":"output2","type":0,"uri":"tmp_3aef3c05ffa440038c4dac859e56276d"},{"name":"output3","type":0,"uri":"tmp_0cac478beda54db795a7dd5348526e53"},{"name":"output4","type":0,"uri":"tmp_e5905eb6dedd4a0d92a2e427b3084426"}]'

# 自定义参数


## 安装

In [32]:
# install fastai
!pip install fastai
print('installation ends')

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
installation ends


## 训练

In [33]:
import wfio
import pandas as pd
from pandas.core.frame import DataFrame

# preprocess
import time
import datetime

from fastai import *
from fastai.tabular import *

from ustciscrBDL_B import get_score

import matplotlib.pyplot as plt
import random

trainset1 = wfio.read_dataframe(_TRAINSET1)
trainset2 = wfio.read_dataframe(_TRAINSET2)
# testseta = wfio.read_dataframe(_TESTSETA)
# testseta['yc'] = 0
testsetb = wfio.read_dataframe(_TESTSETB)
testsetb['yc'] = 0

### 合并trainset 1 2 到 trainset

In [34]:
trainset = pd.DataFrame()
# 前96个税务属性直接拼接
for i in range(96):
    attr = str(trainset2.columns[i])
    trainset[attr] = pd.concat([trainset1[attr], trainset2[attr]])

# 后18个企业属性
t1_attrs = ['cwryxm','cwrysjh',
'cyrs', 'djkyrq',
'frsjh', 'fddbrxm',
'xy', 'xydl', 'xyml', 'xyzl', 'jyfw',
'nsrmc',
'scjydz', 'jdxz',
'yc', 
'zcdz', 'zczby', 'zjnsrsbh'
]
for i in range(96,114):
    t1_attr = t1_attrs[i-96]
    attr = str(trainset2.columns[i])
    trainset[attr] = pd.concat([trainset1[t1_attr], trainset2[attr]])

print(trainset.columns.size)
print(trainset.columns[-1-18:-1])

114
Index(['2019q4rkse', 'bsrxm', 'bsrxmmp', 'cyrs', 'djrq', 'fdbrxmp', 'fddbrxm',
       'hy', 'hydl', 'hyml', 'hyzl', 'jyfw', 'nsrmc', 'scjydz', 'xzjd', 'yc',
       'zcdz', 'zczb'],
      dtype='object')


### 登记日期 转为距今（2019-12-32）多少

In [35]:
# def transform_attr_djrq(df):
#     for index, row in df.iterrows():
#         start = str(row['djrq'])
#         end = '2019-12-31 00:00:00'
#         try:
#             start=time.strptime(start, "%Y-%m-%d %H:%M:%S")
#         except ValueError:
#             start=time.strptime(start, "%Y-%m-%d")
#         end=time.strptime(end, "%Y-%m-%d %H:%M:%S")
#         userStart=datetime.datetime(start[0],start[1],start[2])
#         userEnd=datetime.datetime(end[0],end[1],end[2])
#         res = (userEnd-userStart).days
#         row['djrq'] = res

# datasets = [trainset, testseta, testsetb]
# for dataset in datasets:
#     transform_attr_djrq(dataset)

### 增加属性：开票比例、进项税额、应纳税额、欠税

In [36]:
def add_tax_attrs(df):
    kpbl=[]; jxse=[]; ynse=[]; qs=[]
    for i in range(12):
        kpbl.append([]); jxse.append([]); ynse.append([]); qs.append([])

    for index, row in df.iterrows():
        for yn in range(3):
            for sn in range(4):
                prefix = str(2017+yn)+'q'+str(sn+1)
                i = yn*4+sn # 12季度中的第几季度(从0开始)
                attr_names = []
                for attr_body in ['kps', 'fphdsl',
                    'jxje', 'jxsl',
                    'kpse',
                    'rkse'
                ]: attr_names.append(prefix+attr_body)
                
                try: kps=float(row[attr_names[0]]);
                except: kps=0
                try: fphdsl=float(row[attr_names[1]])
                except: fphdsl=0
                try: jxje=float(row[attr_names[2]])
                except: jxje=0
                try: jxsl=float(row[attr_names[3]])
                except: jxsl=0
                try: kpse=float(row[attr_names[4]])
                except: kpse=0
                try: rkse=float(row[attr_names[5]])
                except: rkse=0
                

                # 开票数
                if fphdsl>0: kpbl[i].append(kps/fphdsl)
                else: kpbl[i].append(kps)
                # 进项税额
                tmp_jxse = jxje * jxsl
                jxse[i].append(tmp_jxse)
                # 应纳税额
                tmp_ynse = kpse - tmp_jxse * (jxsl >= 0.03)
                ynse[i].append(tmp_ynse)
                # 欠税
                tmp_qs = tmp_ynse - rkse
                if tmp_qs>0: qs[i].append(tmp_qs)
                else: qs[i].append(0)

    # 走完所有row
    for yn in range(3):
            for sn in range(4):
                prefix = str(2017+yn)+'q'+str(sn+1)
                i = yn*4+sn # 12季度中的第几季度(从0开始)
                attr_names = []
                for attr_body in ['kpbl', 'jxse', 'ynse', 'qs']:
                    attr_names.append(prefix+attr_body)
                df[attr_names[0]] = kpbl[i]
                df[attr_names[1]] = jxse[i]
                df[attr_names[2]] = ynse[i]
                df[attr_names[3]] = qs[i]

datasets = [trainset, testsetb]
for dset in datasets:
    add_tax_attrs(dset)

### 修正列类型

In [37]:
# 修正列类型
def correct_dtypes(df):
    cols = df.columns
    # 前8*12=96个税务属性
    for i in range(96):
        attr = cols[i]
        df[attr] = pd.to_numeric(df[attr], errors='coerce').fillna(0)
    # 尾4*12=48个税务属性
    for i in range(114,114+48):
        attr = cols[i]
        df[attr] = pd.to_numeric(df[attr], errors='coerce').fillna(0)
    # 企业属性和异常属性
    for attr in ['cyrs', 'djrq', 'yc', 'zczb']:
        df[attr] = pd.to_numeric(df[attr], errors='coerce').fillna(0)

# 查看各列属性
def show_dtypes(df):
    cols = df.columns
    print('--------check column dtype-----------')
    print('--------first 96 tax attrs--------')
    print('take one season (8 attrs) to represent')
    for i in range(8):
        print(cols[i], end='\t')
        print(trainset[cols[i]].dtype)
    print('-----------mid 14 company attrs-----------')
    for i in range(100, 114):
        print(cols[i], end='\t')
        print(trainset[cols[i]].dtype)
    print('-----------last 48 tax attrs-----------')
    print('take one season (4 attrs) to represent')
    for i in range(114,114+4):
        print(cols[i], end='\t')
        print(trainset[cols[i]].dtype)

correct_dtypes(trainset)
show_dtypes(trainset)
correct_dtypes(testsetb)

--------check column dtype-----------
--------first 96 tax attrs--------
take one season (8 attrs) to represent
2017q1fphdsl	int64
2017q1jxje	float64
2017q1jxsl	float64
2017q1kpje	float64
2017q1kps	int64
2017q1kpse	float64
2017q1kpsl	float64
2017q1rkse	float64
-----------mid 14 company attrs-----------
fdbrxmp	object
fddbrxm	object
hy	object
hydl	object
hyml	object
hyzl	object
jyfw	object
nsrmc	object
scjydz	object
xzjd	object
yc	int64
zcdz	object
zczb	float64
zjnsrsbh	object
-----------last 48 tax attrs-----------
take one season (4 attrs) to represent
2017q1kpbl	float64
2017q1jxse	float64
2017q1ynse	float64
2017q1qs	float64


### 模型参数

In [38]:
!mkdir model
!ls

mkdir: cannot create directory ‘model’: File exists
model


In [39]:
# 缺失值，分类值，连续值的处理方法
procs = [FillMissing, Categorify, Normalize]

# 报税人姓名bsrxm
# 法定代表人姓名fddbrxm,
# 行业hy, 行业中类hyzl,
# 纳税人名称nsrmc, 
cat_names3 = [
    'bsrxm',
    'fddbrxm',
    'hy', 'hyzl',
    'nsrmc'
]

# 从业人数 cyrs, 
# 注册资本zczb, 
# 开票数kps，开票比例 kpbl，
# 进项金额jxje，进项税额jxse，
# 开票金额kpje，开票税额kpse，
# 应纳税额ynse，入库金额rkse，欠税qs
cont_names3 = [
    'cyrs',
    'zczb'
]
for yn in range(3): # year num
    for sn in range(4): # seasom num
        prefix = str(2017+yn)+'q'+str(sn+1)
        for attr in [
            'kps', 'fphdsl', 'kpbl',
            'jxje', 'jxsl', 'jxse',
            'kpje', 'kpsl', 'kpse',
            'ynse', 'rkse',  'qs'
        ]:
            attr = prefix + attr
            cont_names3.append(attr)
            
# 因变量
dep_var = 'yc'

path = 'model' # 存放训练过程中产生的数据，如模型

# n = len(trainset)
# valid_idx = range(int(n*0.8), n) # 后20% 4千条作验证集，即trainset2的40%

valid_idx = []
traindata = TabularDataBunch.from_df(path, trainset, dep_var, valid_idx, procs, cat_names3, cont_names3, test_df=testsetb)

learn6 = tabular_learner(traindata, layers=[500,300], ps=[0.001,0.01], emb_drop=0.1, metrics=accuracy)
learn6.model

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(9417, 269)
    (1): Embedding(11288, 298)
    (2): Embedding(576, 56)
    (3): Embedding(312, 40)
    (4): Embedding(16501, 368)
  )
  (emb_drop): Dropout(p=0.1, inplace=False)
  (bn_cont): BatchNorm1d(146, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=1177, out_features=500, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.001, inplace=False)
    (4): Linear(in_features=500, out_features=300, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.01, inplace=False)
    (8): Linear(in_features=300, out_features=2, bias=True)
  )
)

In [40]:
learn6.fit_one_cycle(1, 1e-2, wd=0.2)
learn6.save('model6', return_path=True)

PosixPath('model/models/model6.pth')

## 测试

In [41]:
get_score.post_user_id('18')
preds6, y6 = learn6.get_preds(DatasetType.Test)

tb6 = pd.DataFrame()
tb6['zjnsrsbh'] = testsetb['zjnsrsbh']

# 概率yc[0,1]的分数
pr = preds6.narrow(dim=1, start=1, length=1).tolist()
pr = [x[0] for x in pr]
tb6['Probability'] = pr
print('------------概率yc[0,1]的分数---------------')
get_score.post_verify_data(tb6)

------------概率yc[0,1]的分数---------------
User-id:18
程序开始运行时间为：2019-12-22 14:36:08.313582
程序结束运行时间为：2019-12-22 14:36:09.700410
程序运行时间（去除打分耗时）为：1.386828
程序的准确率为：90.23486265
最终得分为：91.21137638
